In [31]:
import numpy as np
import pandas as pd
import tensorrec
import pickle
from scipy import sparse

In [2]:
rating_df=pd.read_csv("sur_lotion_rating.csv")

In [3]:
rating_df.head()

,user_id,product_id,rating
0,0,43,5
1,1,43,5
2,2,43,5
3,3,43,5
4,4,43,5


In [40]:
rating_df=rating_df[rating_df.groupby('user_id')['user_id'].transform('size') > 1] 

In [48]:
rating_df['user_id'].value_counts()

1354     12
1239     10
508      10
224       9
1138      9
176       8
440       8
2497      8
3314      8
4016      7
1887      7
3606      7
2385      6
3583      6
218       6
268       6
1070      6
921       6
629       6
510       6
225       6
4964      6
537       6
3794      6
8477      6
926       6
734       6
2609      6
619       6
2310      6
         ..
3055      2
5012      2
9106      2
5008      2
2915      2
2883      2
4932      2
4936      2
2895      2
9042      2
11091     2
11191     2
9046      2
7005      2
862       2
4960      2
4968      2
910       2
4972      2
7021      2
11119     2
4976      2
4980      2
886       2
2935      2
4988      2
4996      2
5000      2
5004      2
4         2
Name: user_id, Length: 2176, dtype: int64

In [49]:
rating_df.head()

,user_id,product_id,rating
4,4,43,5
5,4,24,5
6,5,43,5
7,5,1,3
14,12,43,5


In [50]:
from sklearn.model_selection import train_test_split
train, test= train_test_split(rating_df, test_size=0.3, random_state=42)

In [51]:
train2=train.copy()

In [52]:
test2=test.copy()

In [53]:
test['rating']=0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
train['rating']=0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
train1=pd.concat([train2, test])

In [56]:
test1=pd.concat([train, test2])

In [57]:
trainp = train1.pivot(index = 'user_id', columns ='product_id', values = 'rating').fillna(0)

In [58]:
testp = test1.pivot(index = 'user_id', columns ='product_id', values = 'rating').fillna(0)

In [59]:
trainset=sparse.coo_matrix(trainp.values)

In [60]:
testset=sparse.coo_matrix(testp.values)

In [61]:
print(trainset)

  (0, 24)	5.0
  (0, 43)	5.0
  (1, 1)	3.0
  (1, 43)	5.0
  (2, 24)	4.0
  (2, 32)	4.0
  (2, 43)	5.0
  (3, 43)	5.0
  (4, 42)	5.0
  (4, 43)	5.0
  (5, 19)	5.0
  (5, 43)	5.0
  (7, 7)	4.0
  (7, 43)	5.0
  (8, 23)	3.0
  (8, 43)	5.0
  (9, 43)	5.0
  (10, 43)	5.0
  (11, 96)	4.0
  (12, 43)	5.0
  (13, 43)	5.0
  (14, 35)	5.0
  (15, 43)	5.0
  (15, 50)	5.0
  (16, 23)	4.0
  :	:
  (2159, 19)	4.0
  (2160, 19)	4.0
  (2160, 96)	5.0
  (2161, 19)	3.0
  (2161, 72)	3.0
  (2163, 32)	5.0
  (2164, 19)	2.0
  (2164, 72)	3.0
  (2165, 19)	1.0
  (2165, 96)	3.0
  (2166, 32)	4.0
  (2167, 32)	4.0
  (2167, 56)	3.0
  (2168, 56)	3.0
  (2168, 72)	4.0
  (2169, 32)	5.0
  (2169, 68)	3.0
  (2170, 72)	3.0
  (2171, 32)	1.0
  (2171, 72)	1.0
  (2172, 53)	4.0
  (2172, 96)	5.0
  (2173, 72)	5.0
  (2174, 68)	5.0
  (2175, 53)	5.0


In [62]:
testset

<2176x100 sparse matrix of type '<class 'numpy.float64'>'
	with 1583 stored elements in COOrdinate format>

In [70]:
from surprise.model_selection import GridSearchCV

In [95]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.evaluation import recall_at_k
# Instantiate and train the model
model = LightFM(loss='warp',random_state=0,learning_rate= 0.01,no_components= 56)
model.fit(trainset, epochs=11, num_threads=2)

# Evaluate the trained model
test_precision = precision_at_k(model, testset, k=10).mean()

In [84]:
?LightFM

In [96]:
train_auc = auc_score(model, trainset).mean()
test_auc = auc_score(model, testset).mean()

In [97]:
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

AUC: train 0.77, test 0.73.


In [98]:
test_precision = precision_at_k(model, testset, k=10).mean()

In [99]:
test_precision

0.04116203

In [100]:
test_recall = recall_at_k(model, testset, k=10).mean()

In [101]:
test_recall

0.3172122204037098

In [77]:
from skopt import forest_minimize
def objective(params):
    # unpack
    epochs, learning_rate,\
    no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    model = LightFM(loss='warp',
                    random_state=2016,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(trainset, epochs=epochs,
              num_threads=4, verbose=True)
    
    patks = lightfm.evaluation.precision_at_k(model, testset,
                                              train_interactions=None,
                                              k=5, num_threads=4)
    mapatk = np.mean(patks)
    # Make negative because we want to _minimize_ objective
    out = -mapatk
    # Handle some weird numerical shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [79]:
import lightfm

In [80]:
space = [(1, 260), # epochs
         (10**-4, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-6, 10**-1, 'log-uniform'), # alpha
        ]

res_fm = forest_minimize(objective, space, n_calls=250,
                     random_state=0,
                     verbose=True)
print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'alpha']
for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.8450
Function value obtained: -0.0498
Current minimum: -0.0498
Iteration No: 2 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.4350
Function value obtained: -0.0530
Current minimum: -0.0530
Iteration No: 3 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 

Epoch 112
Epoch 113
Epoch 114
Epoch 115
Epoch 116
Epoch 117
Epoch 118
Epoch 119
Epoch 120
Epoch 121
Epoch 122
Epoch 123
Epoch 124
Epoch 125
Epoch 126
Epoch 127
Epoch 128
Epoch 129
Epoch 130
Epoch 131
Epoch 132
Epoch 133
Epoch 134
Epoch 135
Epoch 136
Epoch 137
Epoch 138
Epoch 139
Epoch 140
Epoch 141
Epoch 142
Epoch 143
Epoch 144
Epoch 145
Epoch 146
Epoch 147
Epoch 148
Epoch 149
Epoch 150
Epoch 151
Iteration No: 8 ended. Evaluation done at random point.
Time taken: 18.1350
Function value obtained: -0.0514
Current minimum: -0.0540
Iteration No: 9 started. Evaluating function at random point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45


Epoch 34
Epoch 35
Epoch 36
Iteration No: 16 ended. Search finished for the next optimal point.
Time taken: 0.8620
Function value obtained: -0.0481
Current minimum: -0.0545
Iteration No: 17 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 0.6370
Function value obtained: -0.0447
Current minimum: -0.0545
Iteration No: 18 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40

Epoch 179
Epoch 180
Epoch 181
Epoch 182
Epoch 183
Epoch 184
Epoch 185
Epoch 186
Epoch 187
Epoch 188
Epoch 189
Epoch 190
Epoch 191
Epoch 192
Epoch 193
Epoch 194
Epoch 195
Epoch 196
Epoch 197
Epoch 198
Epoch 199
Epoch 200
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 2.7840
Function value obtained: -0.0484
Current minimum: -0.0545
Iteration No: 23 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.3880
Function value obtained: -0.0430
Current minimum: -0.0545
Iteration No: 24 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epo

Epoch 130
Epoch 131
Epoch 132
Epoch 133
Epoch 134
Epoch 135
Epoch 136
Epoch 137
Epoch 138
Epoch 139
Epoch 140
Epoch 141
Iteration No: 28 ended. Search finished for the next optimal point.
Time taken: 22.5350
Function value obtained: -0.0520
Current minimum: -0.0545
Iteration No: 29 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch

...

Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110
Epoch 111
Epoch 112
Epoch 113
Epoch 114
Epoch 115
Epoch 116
Epoch 117
Epoch 118


Epoch 127
Epoch 128
Epoch 129
Epoch 130
Epoch 131
Epoch 132
Epoch 133
Epoch 134
Iteration No: 234 ended. Search finished for the next optimal point.
Time taken: 8.9270
Function value obtained: -0.0498
Current minimum: -0.0545
Iteration No: 235 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79

Epoch 116
Epoch 117
Epoch 118
Epoch 119
Epoch 120
Epoch 121
Epoch 122
Epoch 123
Epoch 124
Epoch 125
Epoch 126
Epoch 127
Epoch 128
Epoch 129
Epoch 130
Epoch 131
Epoch 132
Epoch 133
Epoch 134
Epoch 135
Epoch 136
Epoch 137
Epoch 138
Epoch 139
Epoch 140
Epoch 141
Epoch 142
Epoch 143
Epoch 144
Epoch 145
Epoch 146
Epoch 147
Epoch 148
Epoch 149
Epoch 150
Epoch 151
Epoch 152
Epoch 153
Epoch 154
Epoch 155
Epoch 156
Epoch 157
Epoch 158
Epoch 159
Epoch 160
Epoch 161
Epoch 162
Epoch 163
Epoch 164
Epoch 165
Epoch 166
Epoch 167
Epoch 168
Epoch 169
Epoch 170
Epoch 171
Epoch 172
Epoch 173
Epoch 174
Iteration No: 238 ended. Search finished for the next optimal point.
Time taken: 5.1270
Function value obtained: -0.0293
Current minimum: -0.0545
Iteration No: 239 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Ep

Epoch 208
Epoch 209
Epoch 210
Epoch 211
Epoch 212
Epoch 213
Epoch 214
Epoch 215
Epoch 216
Epoch 217
Epoch 218
Epoch 219
Epoch 220
Epoch 221
Epoch 222
Epoch 223
Epoch 224
Epoch 225
Epoch 226
Epoch 227
Epoch 228
Epoch 229
Epoch 230
Epoch 231
Epoch 232
Epoch 233
Epoch 234
Epoch 235
Epoch 236
Epoch 237
Epoch 238
Epoch 239
Epoch 240
Epoch 241
Epoch 242
Epoch 243
Epoch 244
Iteration No: 243 ended. Search finished for the next optimal point.
Time taken: 35.4110
Function value obtained: -0.0175
Current minimum: -0.0545
Iteration No: 244 started. Searching for the next optimal point.
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch

Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110
Epoch 111
Epoch 112
Epoch 113
Epoch 114
Epoch 115
Epoch 116
Epoch 117
Epoch 118
Epoch 119
Epoch 120
Epoch 121
Epoch 122
Epoch 123
Epoch 124
Epoch 125
Epoch 126
Epoch 127
Epoch 128
Epoch 129
Epoch 130
Epoch 131
Epoch 132
Epoch 133
Epoch 134
Epoch 135
Epoch 136
Epoch 137
Epoch 138
Epoch 139
Epoch 140
Epoch 141
Epoch 142
Epoch 143
Epoch 144
Epoch 145
Epoch 146
Epoch 147
Epoch 148
Epoch 149
Epoch 150
Epoch 151
Epoch 152
Epoch 153
Epoch 154
Epoch 155
Epoch 156
Epoch 157
Epoch 158
Epoch 159
Epoch 160
Epoch 161
Epoch 162
Epoch 163
Epoch 164
Epoch 165
Epoch 166
Epoch 167
Epoch 168
Epoch 169
Epoch 170
Epoch 171
Epoch 172
Epoch 173
Epoch 174
Epoch 175
Epoch 176
Epoch 177
Epoch 178
Epoch 179
Epoch 180
Epoch 181
Epoch 182
Epoch 183
Epoch 184
Epoch 185
Epoch 186
Epoch 187
Epoch 188
Epoch 189
Epoch 190
